# Árboles de Decisión

## Proyecto 2 - Entrega 2

### Predicción del precio de casas 

El resultado obtenido muestra que ....

### Desarrollo de 3 modelos adicionales

### Creación de variable para clasificar casas en Económicas, Intermedias o Caras.

### Árbol de clasificación con variable respuesta

### Análisis de eficiencia

### Modelo con validación cruzada

### Elaboración de 3 modelos adicionales

### Utilización de Random Forest

### Resultados y Comparaciones